In [7]:
"""국토교통부 아파트매매 실거래 데이터 추출 예시
- 링크: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057511
"""
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from tqdm import tqdm
import xmltodict


# load .env
load_dotenv()
getRTMSDataSvcAptTradeDev_en = os.environ.get("getRTMSDataSvcAptTradeDev_en")
getRTMSDataSvcAptTradeDev_de = os.environ.get("getRTMSDataSvcAptTradeDev_de")

In [27]:
start_date = pd.to_datetime('1980-01-01')
end_date = pd.to_datetime('2022-10-31')
dates = pd.date_range(start_date, end_date, freq="M")

In [28]:
MONTH_LIST = [date.strftime("%Y%m") for date in dates]
RES_ROWS = 10000
SIGUNGU_CODE = 11470

In [29]:
url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev"

In [30]:
df_all = pd.DataFrame()

for month in tqdm(MONTH_LIST):
    params = {
        "serviceKey": getRTMSDataSvcAptTradeDev_de,
        "pageNo": "1",
        "numOfRows": RES_ROWS,
        "LAWD_CD": SIGUNGU_CODE,  # Input 1: 시군구코드 5자리, ex) 11680 = 서울특별시 강남구
        "DEAL_YMD": month,  # Input 2: 년월(YYYYMM), ex) 202208 = 2022년 8월
    }

    response = requests.get(url, params=params)

    if xmltodict.parse(response.content)["response"]["body"]["items"]:
        dict_content = xmltodict.parse(response.content)["response"]["body"]["items"]["item"]
        df = pd.DataFrame(dict_content)
        df_all = pd.concat([df_all, df])

    # df.to_csv(f"data/서울특별시_양천구_아파트매매/서울특별시_양천구_아파트매매_{month}.csv",
    #           encoding="utf8",
    #           index=False
    # )

 95%|█████████▍| 487/514 [03:09<00:10,  2.57it/s]


KeyError: 'body'

In [33]:
df_all

,거래금액,거래유형,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,...,월,일,일련번호,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"33,300",None,1987,2006,목동서로,00340,00000,11470,01,0,...,1,1,11470-125,53.82,None,312,11470,7,None,None
1,"43,000",None,1988,2006,목동동로,00050,00000,11470,01,0,...,1,2,11470-121,71.64,None,326,11470,5,None,None
2,"25,500",None,1987,2006,목동서로,00400,00000,11470,01,0,...,1,3,11470-119,53.82,None,311,11470,15,None,None
3,"13,500",None,1987,2006,신정로13길,00035,00000,11470,01,0,...,1,3,11470-135,48.6,None,733-31,11470,3,None,None
4,"14,000",None,1996,2006,오목로,00232,00000,11470,01,0,...,1,3,11470-130,59.73,None,992-1,11470,9,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,"34,000",None,2004,2020,곰달래로2길,00031,00000,11470,01,0,...,7,30,11470-335,83.55,None,1032,11470,8,None,None
501,"32,700",None,1996,2020,곰달래로2길,00016,00000,11470,01,0,...,7,31,11470-89,77.31,None,222-7,11470,6,None,None
502,"57,400",None,2005,2020,오목로3길,00017,00000,11470,00,0,...,7,31,11470-3945,84.14,None,1041,11470,7,None,None
503,"62,000",None,2007,2020,오목로17길,00007,00000,11470,01,0,...,7,31,11470-4125,84.73,None,1052,11470,8,None,None


In [34]:
df_all.to_csv("data/서울특별시_양천구_아파트매매_1980_202210.csv", index=False)